In [1]:
import pandas as pd
import re
import os
import numpy as np

In [ ]:
from models import syn
exchanges = set(i.id for i in syn.exchanges)
genes = sorted([i.id for i in syn.genes])

In [81]:
old_to_new = dict()
for i in syn.genes:
#     print(i.annotation['refseq_old_locus_tag'])
    if 'refseq_old_locus_tag' in i.annotation:
        old_to_new[i.annotation['refseq_old_locus_tag']] = i.id
    else:
        old_to_new[i.id] = i.id
        print(i.id)

Synpcc7942_2454
Synpcc7942_0034
Synpcc7942_0413
Synpcc7942_1971
Synpcc7942_2107
Synpcc7942_2010
Synpcc7942_1478
Synpcc7942_0491
Synpcc7942_0668


In [83]:
genes[:5]

['SYNPCC7942_RS00005',
 'SYNPCC7942_RS00015',
 'SYNPCC7942_RS00020',
 'SYNPCC7942_RS00045',
 'SYNPCC7942_RS00090']

In [84]:
ge_data = pd.read_excel(
    'data_ge_syn/pnas.1613446113.sd03_GENE_ESSENTIALITY.xlsx',
    skiprows=11,
    sheet_name='Full essentiality list'
)

C:\Users\pino216\Miniconda3\envs\whole-cell-gemm\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [100]:
important_data = ge_data[['Gene ID', 'RB-TnSeq essentiality']].copy()
important_data['gene'] = important_data['Gene ID']
important_data['essential'] = important_data['RB-TnSeq essentiality']
del important_data['RB-TnSeq essentiality']
del important_data['Gene ID']
important_data.head()
important_data['gene'] = important_data['gene'].map(old_to_new)
important_data.dropna(subset=['gene'], inplace=True)

In [101]:
# important_data['gene'] = important_data.gene.str.replace('Synpcc7942_', 'SYNPCC_')

In [102]:
important_data['gene']

0      SYNPCC7942_RS00005
1      SYNPCC7942_RS00015
2      SYNPCC7942_RS00020
3      SYNPCC7942_RS00045
4      SYNPCC7942_RS00090
              ...        
780    SYNPCC7942_RS13310
781    SYNPCC7942_RS13425
782    SYNPCC7942_RS13430
783    SYNPCC7942_RS13375
784    SYNPCC7942_RS13380
Name: gene, Length: 785, dtype: object

In [103]:
data_genes = sorted(important_data['gene'].values)

In [104]:
data_genes[-5:]

['Synpcc7942_1478',
 'Synpcc7942_1971',
 'Synpcc7942_2010',
 'Synpcc7942_2107',
 'Synpcc7942_2454']

In [105]:
print(len(set(data_genes).intersection(genes)))
print(len(set(data_genes).difference(genes)))
print(len(set(genes).difference(data_genes)))

785
0
0


In [106]:
set(genes).difference(data_genes)

set()

In [ ]:
important_data['gene'][important_data['gene'].isin(genes)]

In [108]:
important_data

,gene,essential
0,SYNPCC7942_RS00005,non-essential
1,SYNPCC7942_RS00015,ambiguous
2,SYNPCC7942_RS00020,essential
3,SYNPCC7942_RS00045,essential
4,SYNPCC7942_RS00090,beneficial
...,...,...
780,SYNPCC7942_RS13310,non-essential
781,SYNPCC7942_RS13425,non-essential
782,SYNPCC7942_RS13430,non-essential
783,SYNPCC7942_RS13375,non-essential


In [109]:
important_data.loc[~(important_data.essential == 'essential'), 'essential'] = False
important_data.loc[important_data.essential == 'essential', 'essential'] = True


In [110]:
important_data

,gene,essential
0,SYNPCC7942_RS00005,False
1,SYNPCC7942_RS00015,False
2,SYNPCC7942_RS00020,True
3,SYNPCC7942_RS00045,True
4,SYNPCC7942_RS00090,False
...,...,...
780,SYNPCC7942_RS13310,False
781,SYNPCC7942_RS13425,False
782,SYNPCC7942_RS13430,False
783,SYNPCC7942_RS13375,False


In [111]:
important_data.to_csv('syn_elo_knockdown.csv')